### Çoklu Doğrusal Regresyon

In [1]:
import pandas as pd 
ad = pd.read_csv("Advertising.csv" , usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [8]:
Z = df.drop("sales",axis=1)
y = df["sales"]
#test train nasıl ayrılır
from sklearn.model_selection import train_test_split , cross_val_score , cross_val_predict

x_train , x_test , y_train , y_test = train_test_split(Z, y, test_size = 0.2 , random_state = 42)

x_train.shape

(160, 3)

In [9]:
x_test.shape

(40, 3)

In [10]:
#160 a 40 bölünmüş oldu
training = df.copy()

In [11]:
training.shape

(200, 4)

### statsmodels

In [13]:
import statsmodels.api as sm
lm = sm.OLS(y_train,x_train)
#yorumlama ihtiyacım varsa
model = lm.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  sales   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.982
Method:                 Least Squares   F-statistic:                              2935.
Date:                Thu, 11 Mar 2021   Prob (F-statistic):                   1.28e-137
Time:                        21:52:45   Log-Likelihood:                         -336.65
No. Observations:                 160   AIC:                                      679.3
Df Residuals:                     157   BIC:                                      688.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TV             0.0531      0.001     36.467      0.000       0.050       0.056
radio          0.2188      0.011     20.138      0.000       0.197       0.240
newspaper      0.0239      0.008      3.011      0.003       0.008       0.040
==============================================================================
Omnibus:                       11.405   Durbin-Watson:                   1.895
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               15.574
Skew:                          -0.432   Prob(JB):                     0.000415
Kurtosis:                       4.261   Cond. No.                         13.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
model.summary().tables[1]
#sadece katsayıları verir

,coef,std err,t,P>|t|,[0.025,0.975]
TV,0.0531,0.001,36.467,0.000,0.050,0.056
radio,0.2188,0.011,20.138,0.000,0.197,0.240
newspaper,0.0239,0.008,3.011,0.003,0.008,0.040


### Scikit model

In [16]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(x_train,y_train)

In [17]:
model.intercept_

2.979067338122629

In [18]:
model.coef_

array([0.04472952, 0.18919505, 0.00276111])

### Tahmin

In [19]:
 # Sales = 2.97 + TV * 0.04 + Newspaper * 0.002
#30 tv harcaması 10 radio 40 gazete harcaması olsa tahmini satış değeri ne olur ? 
yeniveri = [[30],[10],[40]]
yeniveri = pd.DataFrame(yeniveri).T


In [20]:
model.predict(yeniveri)

array([6.32334798])

In [24]:
#modelimizin tahmin basarısı nedir?
from sklearn.metrics import mean_squared_error,r2_score
import numpy as np
rmse = np.sqrt(mean_squared_error(y_train,model.predict(x_train)))

In [25]:
rmse

1.644727765644337

In [26]:
#test hatası nedir
rmse = np.sqrt(mean_squared_error(y_test,model.predict(x_test)))

In [27]:
rmse

1.7815996615334502

### Model Tuning - Model Dogrulama

In [29]:
np.sqrt(mean_squared_error(y_train,model.predict(x_train)))

1.644727765644337

In [30]:
model.score(x_train,y_train)

0.8957008271017818

In [32]:
#modelin hangi belirli kısmını secmeliyiz? 
cross_val_score(model,Z,y,cv = 10 ,scoring="r2").mean()

0.8853562237979616

In [33]:
cross_val_score(model,x_train,y_train,cv = 10 ,scoring="r2").mean()

0.791354859691634

In [38]:
np.sqrt(-cross_val_score(model,x_train,y_train,cv = 10 ,scoring="neg_mean_squared_error")).mean()
#gercek egitim hatası

1.6513523730313335

In [40]:
np.sqrt(mean_squared_error(y_test,model.predict(x_test)))

1.7815996615334502

In [42]:
#gercek test hatası
np.sqrt(-cross_val_score(model,x_test,y_test,cv = 10 ,scoring="neg_mean_squared_error")).mean()

1.8462778823997095

In [ ]:
#train hatası ile test hatası birbirinden farklılaşıyor
#eğitim hatası test hatasının kötü bir tahmincisi
#valide edilmemiş hata da kötüdür
